In [1]:
import pandas as pd
from datasets import load_dataset
import os
if not os.path.exists("../data/train.csv"):
    ds = load_dataset("traintogpb/aihub-koja-translation-integrated-large-4.3m")
    train = ds['train'].to_pandas()
    valid = ds['validation'].to_pandas()
    test = ds['test'].to_pandas()
    train.to_csv("../data/train.csv", index=False)
    valid.to_csv("../data/valid.csv", index=False)
    test.to_csv("../data/test.csv", index=False)
else:
    print("data already exists")
    train = pd.read_csv("../data/train.csv")
    valid = pd.read_csv("../data/valid.csv")
    test = pd.read_csv("../data/test.csv")

data already exists


In [2]:
from konlpy.tag import Okt
import openai
import requests
from fugashi import Tagger
import sacrebleu
from dotenv import load_dotenv
import os

load_dotenv()

def ja_tokenizing(sentence="私は学校に行きます。"):
    tagger = Tagger()  # 기본 설정, unidic-lite 사전 사용
    tokens = [word.surface for word in tagger(sentence)]
    return tokens


def ko_tokenizing(results):
    okt = Okt()
    tokenized = []
    for result in results:
        tokens_ko = okt.morphs(result['ko'])
        tokens_tr = okt.morphs(result['translation'])
        tokenized.append((tokens_ko, tokens_tr))
    return tokenized

deepl_api_key = os.getenv("DEEPL_API_KEY")

# deepL API

def translate_with_deepL(message):
    url_for_deepL = "https://api-free.deepl.com/v2/translate"
    params = {'auth_key': deepl_api_key,  # 여기에 본인의 DeepL API 키를 입력하세요
              'text': message,
              'source_lang': 'JA',
              'target_lang': 'KO'}
    response = requests.post(url_for_deepL, data=params)
    if response.status_code == 200:
        translation = response.json()['translations'][0]['text']
        return translation
    else:
        print(f"DeepL API 요청 실패: {response.status_code}, {response.text}")
        return None

openai_api_key = os.getenv("OPENAI_API_KEY")

# GPT API
def translate_with_gpt(message):
    client = openai.OpenAI(api_key=openai_api_key)
    response = client.chat.completions.create(
        model="gpt-4.1-nano-2025-04-14",  # gpt-4o 또는 최신 지원 모델명 사용
        messages=[
            {"role": "system", "content": "앞으로 당신은 일본어를 한국어로 번역하는 역할을 합니다. 번역은 자연스럽고 정확해야 합니다."},
            {"role": "user", "content": message}
        ]
    )
    return response.choices[0].message.content

def get_scores(tokenized):
    hypothesis = []
    references = []
    for tokens_ko, tokens_tr in tokenized:
        hypothesis.append(" ".join(tokens_tr))
        references.append(" ".join(tokens_ko))
    
    bleu = sacrebleu.corpus_bleu(hypothesis, [references])
    chrf = sacrebleu.corpus_chrf(hypothesis, [references])
    ter = sacrebleu.corpus_ter(hypothesis, [references])
    return bleu, chrf, ter


In [10]:
import numpy as np
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

np.random.seed(42)
test_sample = test.sample(frac=1, random_state=42).reset_index(drop=True)
# models = [translate_with_gpt, translate_with_deepL]

"""
    facebook/nllb-200-distilled-600M
    OpenNMT/nllb-200-distilled-1.3B-ct2-int8
    Babelscape/mrebel-large
    facebook/nllb-200-1.3B
facebook/nllb-200-3.3B
    facebook/mbart-large-50-many-to-many-mmt
winninghealth/WiNGPT-Babel-2-GGUF
    facebook/nllb-200-distilled-1.3B
google/madlad400-3b-mt
"""

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

results = []
for i in range(200):
    print(f"test_sample['ko'][{i}]: {test_sample['ko'][i]}")
    print(f"test_sample['ja'][{i}]: {test_sample['ja'][i]}")
    # result = models[0](test_sample['ja'][i])
    inputs = tokenizer(test_sample['ja'][i], return_tensors="pt").to(device)
    translated_tokens = model.generate(
        **inputs, forced_bos_token_id=tokenizer.convert_tokens_to_ids("kor_Hang"), max_length=50
    )
    result = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    print(f"번역 결과: {result}\n")
    results.append({
        "ko": test_sample['ko'][i],
        "ja": test_sample['ja'][i],
        "translation": result
    })
    
tokenized = ko_tokenizing(results)
bleu, chrf, ter = get_scores(tokenized)
print(f"전체 샘플 BLEU: {bleu.score:.2f}")
print(f"전체 샘플 CHRF: {chrf.score:.2f}")
print(f"전체 샘플 TER: {ter.score:.2f}")

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

ValueError: Converting from SentencePiece and Tiktoken failed, if a converter for SentencePiece is available, provide a model path with a SentencePiece tokenizer.model file.Currently available slow->fast converters: ['AlbertTokenizer', 'BartTokenizer', 'BarthezTokenizer', 'BertTokenizer', 'BigBirdTokenizer', 'BlenderbotTokenizer', 'CamembertTokenizer', 'CLIPTokenizer', 'CodeGenTokenizer', 'ConvBertTokenizer', 'DebertaTokenizer', 'DebertaV2Tokenizer', 'DistilBertTokenizer', 'DPRReaderTokenizer', 'DPRQuestionEncoderTokenizer', 'DPRContextEncoderTokenizer', 'ElectraTokenizer', 'FNetTokenizer', 'FunnelTokenizer', 'GPT2Tokenizer', 'HerbertTokenizer', 'LayoutLMTokenizer', 'LayoutLMv2Tokenizer', 'LayoutLMv3Tokenizer', 'LayoutXLMTokenizer', 'LongformerTokenizer', 'LEDTokenizer', 'LxmertTokenizer', 'MarkupLMTokenizer', 'MBartTokenizer', 'MBart50Tokenizer', 'MPNetTokenizer', 'MobileBertTokenizer', 'MvpTokenizer', 'NllbTokenizer', 'OpenAIGPTTokenizer', 'PegasusTokenizer', 'Qwen2Tokenizer', 'Realm

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("OpenNMT/nllb-200-distilled-1.3B-ct2-int8")
model = AutoModelForSeq2SeqLM.from_pretrained("OpenNMT/nllb-200-distilled-1.3B-ct2-int8")

In [8]:
import numpy as np
import pandas as pd
import torch

np.random.seed(42)
test_sample = test.sample(frac=1, random_state=42).reset_index(drop=True)
models = [translate_with_gpt, translate_with_deepL]

results = []
for i in range(200):
    print(f"test_sample['ko'][{i}]: {test_sample['ko'][i]}")
    print(f"test_sample['ja'][{i}]: {test_sample['ja'][i]}")
    result = models[1](test_sample['ja'][i])
    print(f"번역 결과: {result}\n")
    results.append({
        "ko": test_sample['ko'][i],
        "ja": test_sample['ja'][i],
        "translation": result
    })
    
tokenized = ko_tokenizing(results)
bleu, chrf, ter = get_scores(tokenized)
print(f"전체 샘플 BLEU: {bleu.score:.2f}")
print(f"전체 샘플 CHRF: {chrf.score:.2f}")
print(f"전체 샘플 TER: {ter.score:.2f}")

test_sample['ko'][0]: 결과는 조직 내 구성원 간의 경쟁은 항상 긍정적인 결과를 가져오는 것이 아니라 특정한 조건 하에서 긍정적으로 작용한다는 것을 시사한다.
test_sample['ja'][0]: 結果は組織内構成員間の競争は常に肯定的な結果をもたらすのではなく、特定の条件下で肯定的に作用することを示唆する。
번역 결과: 결과는 조직 내 구성원 간의 경쟁이 항상 긍정적인 결과를 가져오는 것이 아니라 특정 조건 하에서 긍정적으로 작용할 수 있음을 시사한다.

test_sample['ko'][1]: 좋은 소리나게 만들어서 보내주세요~
test_sample['ja'][1]: いい音が出るようにして送ってください～
번역 결과: 좋은 소리가 나도록 보내주세요~!

test_sample['ko'][2]: 이 이슈에 대해 보수와 진보의 입장은 갈린다.
test_sample['ja'][2]: この問題について保守と進歩の立場は分かれる。
번역 결과: 이 문제에 대해 보수와 진보의 입장이 갈린다.

test_sample['ko'][3]: 광주시의 경우 전체 행정구역에도시지역과 같은 시가지보다 미 개발지면적이 많으며, 용인시의 경우에는 도시 개발에 따라 시가지면적이 높으며, 도시지역내 녹지면적 비율도 높은 것으로 확인되었다.
test_sample['ja'][3]: 光州市の場合、全行政区域にも市地域のような市街地よりも未開発地面積が多く、龍仁市の場合には都市開発によって市街地面積が高く、都市地域内の緑地面積の比率も高いことが確認された。
번역 결과: 광주의 경우 전체 행정구역에서도 시 지역과 같은 시가지보다 미개발지 면적이 많고, 용인시의 경우 도시개발로 인해 시가지 면적이 높고, 시가지 내 녹지 면적 비율도 높은 것으로 확인되었다.

test_sample['ko'][4]: 또 사람들이 이용하는 기차나 전차에 조사원을 보내 상황을 파악하게 하는 신문사도 있었다.
test_sample['ja'][4]: また人々が利用する汽車や電車に調査員を送って状況を把握させる新聞社もあった。
번역 결과: 또한 사람들이 이용

That's 100 lines that end in a tokenized period ('.')
It looks like you forgot to detokenize your test data, which may hurt your score.
If you insist your data is detokenized, or don't care, you can suppress this message with the `force` parameter.


전체 샘플 BLEU: 51.74
전체 샘플 CHRF: 60.38
전체 샘플 TER: 32.85


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM3-3B")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM3-3B")
messages = [
    {"role": "user", "content": "Who are you?"},
    {}
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))